# Setup

In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nikunjmadan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [67]:
import pandas as pd
import numpy as np
import os
import json
import pickle

from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
os.listdir()

['.DS_Store',
 'Movie Rec.ipynb',
 'tmdb_5000_credits.csv',
 'tmdb_5000_movies.csv',
 '.ipynb_checkpoints']

# Data Exploration

Exploring the Credits Dataset

In [4]:
df_credits = pd.read_csv("tmdb_5000_credits.csv")

In [5]:
df_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [6]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [7]:
df_credits.isna().sum()

movie_id    0
title       0
cast        0
crew        0
dtype: int64

In [8]:
for i in df_credits.columns.to_list():
    print(df_credits[i].value_counts())
    print("--"*50)

movie_id
19995     1
333355    1
71157     1
43418     1
11588     1
         ..
13156     1
293863    1
16871     1
307081    1
25975     1
Name: count, Length: 4803, dtype: int64
----------------------------------------------------------------------------------------------------
title
The Host                 2
Batman                   2
Out of the Blue          2
Avatar                   1
The Girl on the Train    1
                        ..
Step Up 3D               1
Secondhand Lions         1
The Age of Adaline       1
Drag Me to Hell          1
My Date with Drew        1
Name: count, Length: 4800, dtype: int64
----------------------------------------------------------------------------------------------------
cast
[]                                                                                                                                                                                                                                                                           

In [9]:
# exploring cast column - json format of whole cast of the film

cast = json.loads(df_credits.iloc[0]["cast"])
cast[0]

{'cast_id': 242,
 'character': 'Jake Sully',
 'credit_id': '5602a8a7c3a3685532001c9a',
 'gender': 2,
 'id': 65731,
 'name': 'Sam Worthington',
 'order': 0}

In [10]:
# exploring crew column - json format of the whole crew info/

crew = json.loads(df_credits.iloc[0]["crew"])
crew

[{'credit_id': '52fe48009251416c750aca23',
  'department': 'Editing',
  'gender': 0,
  'id': 1721,
  'job': 'Editor',
  'name': 'Stephen E. Rivkin'},
 {'credit_id': '539c47ecc3a36810e3001f87',
  'department': 'Art',
  'gender': 2,
  'id': 496,
  'job': 'Production Design',
  'name': 'Rick Carter'},
 {'credit_id': '54491c89c3a3680fb4001cf7',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Sound Designer',
  'name': 'Christopher Boyes'},
 {'credit_id': '54491cb70e0a267480001bd0',
  'department': 'Sound',
  'gender': 0,
  'id': 900,
  'job': 'Supervising Sound Editor',
  'name': 'Christopher Boyes'},
 {'credit_id': '539c4a4cc3a36810c9002101',
  'department': 'Production',
  'gender': 1,
  'id': 1262,
  'job': 'Casting',
  'name': 'Mali Finn'},
 {'credit_id': '5544ee3b925141499f0008fc',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner'},
 {'credit_id': '52fe48009251416c750ac9c3',
  'department': 'Directing',
  

In [11]:
# df_credits dataset insights:
# movie_id, name of movie(title), cast of movie, crew of movie
# cast and crew columns are explored above too
# no null values

Now exploring Movies Dataset

In [12]:
df_movies = pd.read_csv("tmdb_5000_movies.csv")

In [13]:
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [14]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [15]:
df_movies.isna().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [16]:
# id col in this dataset is same as the movie_id column in credits dataset
# Just some homepage links and taglies of movies are missing, rest data has some minor missing values

In [17]:
explore_cols = ["genres", "keywords", "production_companies", "production_countries", "spoken_languages"]

for i in explore_cols:
    temp = json.loads(df_movies.iloc[0][i])
    
    print(i)
    print(temp)
    print("--"*50)

genres
[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}]
----------------------------------------------------------------------------------------------------
keywords
[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'space war'}, {'id': 3388, 'name': 'space colony'}, {'id': 3679, 'name': 'society'}, {'id': 3801, 'name': 'space travel'}, {'id': 9685, 'name': 'futuristic'}, {'id': 9840, 'name': 'romance'}, {'id': 9882, 'name': 'space'}, {'id': 9951, 'name': 'alien'}, {'id': 10148, 'name': 'tribe'}, {'id': 10158, 'name': 'alien planet'}, {'id': 10987, 'name': 'cgi'}, {'id': 11399, 'name': 'marine'}, {'id': 13065, 'name': 'soldier'}, {'id': 14643, 'name': 'battle'}, {'id': 14720, 'name': 'love affair'}, {'id': 165431, 'name': 'anti war'}, {'id': 193554, 'name': 'power relations'}, {'id': 206690, 'name': 'mind and soul'}, {'id': 209714, 'name': '3d'}]
------------

# Data Pre-processing

In [18]:
len(df_movies)

4803

In [19]:
len(df_credits)

4803

In [20]:
# merging the 2 dataframes

df_merge = df_movies.merge(df_credits, left_on = ["id", "title"], right_on = ["movie_id", "title"], how = "left")
df_merge.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [21]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [22]:
len(df_merge)

4803

In [23]:
# checking merge 
# -1 as title is a common name in both dfs

len(df_movies.columns) + len(df_credits.columns) - 1

23

In [24]:
# Choosing only the relevant / required columns for RECOMMENDATION
# cast and crew is imp as ppl tend to watch their favoutite director or actors

req_cols = ["id", "title", "genres", "keywords", "overview", "cast", "crew"]

df = df_merge[req_cols]
df.head()

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [25]:
# checking duplicate values

df.duplicated().any()

False

handling null values

In [26]:
len(df)

4803

In [27]:
df.isna().sum()

id          0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

In [28]:
# we simple drop them because there are less amt of na values only

df = df.dropna()
len(df)

4800

Processing genres, cast and crew columns and simplifying them

In [29]:
cols = ["genres", "cast", "crew", "keywords"]

for i in range(4):
    print(cols[i])
    temp = json.loads(df.iloc[0][cols[i]])
    if (i == 1 or i == 2):
        print(temp[0])
    else:
        print(temp)
    print("--"*50)

genres
[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}]
----------------------------------------------------------------------------------------------------
cast
{'cast_id': 242, 'character': 'Jake Sully', 'credit_id': '5602a8a7c3a3685532001c9a', 'gender': 2, 'id': 65731, 'name': 'Sam Worthington', 'order': 0}
----------------------------------------------------------------------------------------------------
crew
{'credit_id': '52fe48009251416c750aca23', 'department': 'Editing', 'gender': 0, 'id': 1721, 'job': 'Editor', 'name': 'Stephen E. Rivkin'}
----------------------------------------------------------------------------------------------------
keywords
[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'space war'}, {'id': 3388, 'name': 'space colony'}, {'id': 3679, 'name': 'society'}, {'id': 3801, 'name': 'space travel'}, {'id': 9685, 'name': 'futuristic

In [30]:
# exploring crew / cast col for 1st movie

temp = json.loads(df.iloc[0]["crew"])

print(temp[0].keys())

for i in range(len(temp)):
    if temp[i]["job"] == "Director":
        print(temp[i])
        
temp = json.loads(df.iloc[0]["cast"])

print("/n")
print(temp[0].keys())
for i in temp:
    print(i)
    print("--"*50)

dict_keys(['credit_id', 'department', 'gender', 'id', 'job', 'name'])
{'credit_id': '52fe48009251416c750ac9c3', 'department': 'Directing', 'gender': 2, 'id': 2710, 'job': 'Director', 'name': 'James Cameron'}
/n
dict_keys(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order'])
{'cast_id': 242, 'character': 'Jake Sully', 'credit_id': '5602a8a7c3a3685532001c9a', 'gender': 2, 'id': 65731, 'name': 'Sam Worthington', 'order': 0}
----------------------------------------------------------------------------------------------------
{'cast_id': 3, 'character': 'Neytiri', 'credit_id': '52fe48009251416c750ac9cb', 'gender': 1, 'id': 8691, 'name': 'Zoe Saldana', 'order': 1}
----------------------------------------------------------------------------------------------------
{'cast_id': 25, 'character': 'Dr. Grace Augustine', 'credit_id': '52fe48009251416c750aca39', 'gender': 1, 'id': 10205, 'name': 'Sigourney Weaver', 'order': 2}
--------------------------------------------------------

In [31]:
# creating func to create a list of genres and keywords (handling genre col and keyword col)

def get_genre_list(text):
    result = []
    list = json.loads(text)
    for i in list:
        result.append(i["name"])

    return result

def get_keyword_list(text):
    result = []
    list = json.loads(text)
    for i in list:
        result.append(i["name"])

    return result

In [32]:
# updating the genre col

df["genres"] = df["genres"].apply(get_genre_list)
df["keywords"] = df["keywords"].apply(get_keyword_list)

In [33]:
# Extracting the Director from crew - as ppl tend to watch movies made by their fav. directors too
# there can be multiple directors too

def get_director(text):
    result = []
    list = json.loads(text)
    for i in list:
        if i["job"] == "Director":
            result.append(i["name"])
    return result

In [34]:
df["directors"] = df["crew"].apply(get_director)

In [35]:
# Extracting the top 10 actors from the cast crew -- based on order key
# people often like to watch a movie from their fav. actors as well
# we have taken sample size of 10 because usually the favourite actors are mentioned first in the credits

def get_actors(text):
    result = []
    list = json.loads(text)
    for i in list:
        if i["order"] < 10:
            result.append(i["name"])

    return result

In [36]:
df["actor"] = df["cast"].apply(get_actors)

In [37]:
df["actor"][0]

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez',
 'Giovanni Ribisi',
 'Joel David Moore',
 'CCH Pounder',
 'Wes Studi',
 'Laz Alonso']

In [38]:
df.drop(columns = ["cast", "crew"], inplace=True)

In [39]:
# converting overview column into list -- by splitting based on whitespace

def split_comma(text):
    return text.split(" ")

df["overview"] = df["overview"].apply(split_comma)

In [40]:
# handling names in directors and actor format -- such that FN LN = FNLN -- acting as a single unique name
# this handles having multiple similar first names

def handle_spaces(list):
    result = []
    for i in list:
        result.append(i.replace(" ", ""))
    return result

In [41]:
df["directors"] = df["directors"].apply(handle_spaces)
df["actor"] = df["actor"].apply(handle_spaces)

# similarly handling spaces in other req cols
df["genres"] = df["genres"].apply(handle_spaces)
df["keywords"] = df["keywords"].apply(handle_spaces)

In [42]:
df.head()

,id,title,genres,keywords,overview,directors,actor
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...",[JamesCameron],"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[Captain, Barbossa,, long, believed, to, be, d...",[GoreVerbinski],"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste..."
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[A, cryptic, message, from, Bond’s, past, send...",[SamMendes],"[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[Following, the, death, of, District, Attorney...",[ChristopherNolan],"[ChristianBale, MichaelCaine, GaryOldman, Anne..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[John, Carter, is, a, war-weary,, former, mili...",[AndrewStanton],"[TaylorKitsch, LynnCollins, SamanthaMorton, Wi..."


Concatenating these required information and creating TAGS

In [43]:
df["tags"] = df["genres"] + df["keywords"] + df["overview"] + df["directors"] + df["actor"]

In [44]:
df["tags"][0]

['Action',
 'Adventure',
 'Fantasy',
 'ScienceFiction',
 'cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d',
 'In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.',
 'JamesCameron',
 'SamWorthington',
 'ZoeSaldana',
 'SigourneyWeaver',
 'StephenLang',
 'MichelleRodriguez',
 'GiovanniRibisi',
 'JoelDavidMoore',
 'CCHPounder',
 'WesStudi',
 'LazAlonso']

In [45]:
# Extracting the imp. info

result_df = df[["id", "title", "tags"]]

In [46]:
# converting the tags into a simple paragraph -- for processing -- such that we get a tags list separated by (space)
# also converting all text into lower case

# result_df["tags"] = result_df["tags"].apply(lambda x: " ".join(x.lower()))
def processing(list):
    result = list[0].lower()
    for i in range(1, len(list)):
        result = result + " " + list[i].lower()

    return result

In [47]:
result_df["tags"] = result_df["tags"].apply(processing)

/var/folders/p4/lxvqlf811r948l5gbcpjkhpr0000gn/T/ipykernel_1644/3690708823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["tags"] = result_df["tags"].apply(processing)


In [48]:
result_df.head()

,id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


Stemming to provide consistent words in tags -- Eg: Reducing "action" "actions" to a single token -- "action"

In [49]:
porter = PorterStemmer()

In [50]:
def stemming(text):
    result = []
    for i in text.split():
        token = porter.stem(i)
        result.append(token)

    # output should be a paragraph
    result = " ".join(result)
    
    return result

In [51]:
result_df["tags"] = result_df["tags"].apply(stemming)

/var/folders/p4/lxvqlf811r948l5gbcpjkhpr0000gn/T/ipykernel_1644/1184069295.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["tags"] = result_df["tags"].apply(stemming)


In [52]:
# we would also need to remove stop words -- but that can be removed inside the vectorizer itself

# Recommendation System

In [53]:
result_df

,id,title,tags
0,19995,Avatar,action adventur fantasi sciencefict culturecla...
1,285,Pirates of the Caribbean: At World's End,adventur fantasi action ocean drugabus exotici...
2,206647,Spectre,action adventur crime spi basedonnovel secreta...
3,49026,The Dark Knight Rises,action crime drama thriller dccomic crimefight...
4,49529,John Carter,action adventur sciencefict basedonnovel mar m...
...,...,...,...
4798,9367,El Mariachi,action crime thriller unitedstates–mexicobarri...
4799,72766,Newlyweds,comedi romanc a newlyw couple' honeymoon is up...
4800,231617,"Signed, Sealed, Delivered",comedi drama romanc tvmovi date loveatfirstsig...
4801,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


In [54]:
# Now we need to generate vectors - using counter vectorizer -- 
# all rows tags will be combined together and then generate a Master Paragraph -- ip to counter vectorizer
# then it identifies the most frequently occuring words
# then it compares with the tags of each row -- and identify how many times a particular tag is occured (out of master para??)
# and hence create a vector based on the frequency for each row
# these vectors ofr each row can then be plotted and similar movies can be found out using COSINE SIMILARITY of vectors
# based on the idea that the similar movies will be close to each other on the plot

Creating the vectors - contradicting to the above comment - we use TF-IDF Vectorizer

In [55]:
tfidf_vec = TfidfVectorizer(max_features=5000, stop_words="english")

In [56]:
vector = tfidf_vec.fit_transform(result_df["tags"])
vector

<4800x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 151789 stored elements in Compressed Sparse Row format>

Finding Cosine Similarity

In [57]:
similarity = cosine_similarity(vector)

In [58]:
similarity

array([[1.        , 0.01982677, 0.02832588, ..., 0.02440561, 0.        ,
        0.        ],
       [0.01982677, 1.        , 0.01093668, ..., 0.01627114, 0.        ,
        0.00562515],
       [0.02832588, 0.01093668, 1.        , ..., 0.01612955, 0.        ,
        0.        ],
       ...,
       [0.02440561, 0.01627114, 0.01612955, ..., 1.        , 0.02187599,
        0.0291936 ],
       [0.        , 0.        , 0.        , ..., 0.02187599, 1.        ,
        0.02117459],
       [0.        , 0.00562515, 0.        , ..., 0.0291936 , 0.02117459,
        1.        ]])

Creating the Recommendation function

In [59]:
def make_recommendation(movie_title, df, similarity_scores, n = 10):
    
    # making sure we have ordered index
    df = df.reset_index()
    
    # finding the index of the movie
    try:
        index = df[df["title"] == movie_title].index[0]
    except Exception as e:
        print(f"Incorrect movie name! Movie {movie_title} not present in dataset.")
        return None

    # extracting the similarity score with the above extracted index (for the ip movie)
    # creating a sorted list of scores
    
    scores = list(enumerate(similarity_scores[index]))

    # here key argument referes to which value of the enumerated key do you want to sort on
    scores = sorted(scores, reverse = True, key = lambda x: x[1])

    print("Movie Recommendations:")
    print("--"*40)

    # extracting top n+1 values
    # +1 because it will include the movie itself
    scores = scores[:n+1]
    num = 1
    for i in scores:
        if df["title"][i[0]] != movie_title:
            print(f"Recommendation {num}: {df['title'][i[0]]}")
            print(f"Similarity Score : {round(i[1], 2)}")
            num += 1
    

In [60]:
make_recommendation("Avatar", result_df, similarity)

Movie Recommendations:
--------------------------------------------------------------------------------
Recommendation 1: Falcon Rising
Similarity Score : 0.2
Recommendation 2: Aliens
Similarity Score : 0.2
Recommendation 3: Aliens vs Predator: Requiem
Similarity Score : 0.19
Recommendation 4: Battle: Los Angeles
Similarity Score : 0.19
Recommendation 5: Jarhead
Similarity Score : 0.18
Recommendation 6: Apollo 18
Similarity Score : 0.18
Recommendation 7: Star Trek Into Darkness
Similarity Score : 0.16
Recommendation 8: Meet Dave
Similarity Score : 0.15
Recommendation 9: Lifeforce
Similarity Score : 0.15
Recommendation 10: Titan A.E.
Similarity Score : 0.15


In [61]:
make_recommendation("Spider-Man", result_df, similarity)

Movie Recommendations:
--------------------------------------------------------------------------------
Recommendation 1: Spider-Man 3
Similarity Score : 0.54
Recommendation 2: Spider-Man 2
Similarity Score : 0.48
Recommendation 3: Arachnophobia
Similarity Score : 0.3
Recommendation 4: The Amazing Spider-Man 2
Similarity Score : 0.26
Recommendation 5: The Amazing Spider-Man
Similarity Score : 0.24
Recommendation 6: Eight Legged Freaks
Similarity Score : 0.19
Recommendation 7: Charlotte's Web
Similarity Score : 0.16
Recommendation 8: The Evil Dead
Similarity Score : 0.14
Recommendation 9: X-Men
Similarity Score : 0.13
Recommendation 10: Deadpool
Similarity Score : 0.13


In [62]:
make_recommendation("The Dark Knight Rises", result_df, similarity)

Movie Recommendations:
--------------------------------------------------------------------------------
Recommendation 1: The Dark Knight
Similarity Score : 0.47
Recommendation 2: Batman Returns
Similarity Score : 0.37
Recommendation 3: Batman Begins
Similarity Score : 0.32
Recommendation 4: Batman Forever
Similarity Score : 0.29
Recommendation 5: Batman
Similarity Score : 0.25
Recommendation 6: Batman: The Dark Knight Returns, Part 2
Similarity Score : 0.23
Recommendation 7: Batman & Robin
Similarity Score : 0.22
Recommendation 8: Batman v Superman: Dawn of Justice
Similarity Score : 0.2
Recommendation 9: Slow Burn
Similarity Score : 0.18
Recommendation 10: Inception
Similarity Score : 0.17


In [63]:
make_recommendation("The Dark Knig", result_df, similarity)

Incorrect movie name! Movie The Dark Knig not present in dataset.


# Saving Required Files for Streamlit App

In [65]:
result_df.to_pickle("data/movies_data.pkl")

In [68]:
pickle.dump(similarity, open("data/similarity_matrix.pkl", "wb"))

In [2]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 11.7 MB/s  0:00:001.7 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.4/795.4 kB 10.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 11.5 MB/s  0:00:002.0 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 11.2 MB/s  0:00:03 11.2 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0  0/12 [typing-extensions]
   ━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━  4/12 [pyarrow]enacity]ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/Library/Frameworks/Python.framework/Versions/3.11/etc/jupyter/nbconfig/notebook.d/pydeck.json'
Consider using the `--user` option or check the permissions.

   ━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━  7/12 [pydeck]

[notice] A new release of pip is ava

In [4]:
!pip freeze > requirements.txt